## Setup

In [2]:
%pip install geopandas
%pip install geopy
%pip install gmaps
%pip install pyarrow
%pip install pandas-geojson
%pip install python-google-places
%pip install googlemaps
%pip install ipyleaflet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [60]:
import json
import datetime
import pandas as pd
import googlemaps
from pandas_geojson import to_geojson
from ipyleaflet import Map, GeoJSON, FullScreenControl, Marker, MarkerCluster, AwesomeIcon
from ipywidgets import Layout, HTML

cfg = json.load(open('secure/config.json'))
gmaps = googlemaps.Client(key=cfg['google_maps_api_key'])

## Load school list and add GPS coordinates

In [4]:
df = pd.read_csv('schulsuche_export.csv', delimiter=';', encoding='ISO-8859-1')

In [5]:
df.Schultyp.unique()

array(['Gymnasium', 'Realschule',
       'Volksschule zur sonderp?dagog. F?rderung',
       'Sonstige Berufsfachschule',
       'Berufsfachschule des Gesundheitswesens', 'Gewerbliche Fachschule',
       'Berufsfachschule f. Fremdsprachenberufe',
       'Kaufm?nnische Berufsfachschule', 'Grund- u. Mittelschule',
       'Fachoberschule', 'Berufsoberschule', 'Wirtschaftsschule',
       'Hauswirtschaftliche Fachschule', 'Berufsschule',
       'Berufsfachs. f. Hauswirtschaft u. Sozialberufe', 'Fachakademie',
       'Berufsschule zur sonderp?dagog. F?rderung',
       'Gewerbliche Berufsfachschule',
       'Berufsfachschule f. techn. Assistenzberufe', 'Abendrealschule',
       nan, 'Freie Waldorfschule',
       'Realschule zur sonderp?dagog. F?rderung', 'Ausl?ndische Schule',
       'Schule besonderer Art', 'Berufsfachschule f. Musik',
       'Abendgymnasium', 'Kolleg', 'Sonstige Fachschule',
       'Landwirtschaftliche Fachschule',
       'Fachakademie f?r Landwirtschaft', 'Kaufm?nnische Fac

In [6]:
schools_of_interest = pd.concat([df[df.Schultyp=='Gymnasium'], df[df.Schultyp=='Freie Waldorfschule']], ignore_index=True)

In [7]:
schools_of_interest

,Schulnummer,Schultyp,Name,Strasse,PLZ,Ort,Homepage,Link
0,13,Gymnasium,Kronberg-Gymnasium Aschaffenburg,Fasaneriestr. 33,63739,Aschaffenburg,kronberg-gymnasium.de,/schule/0013.html
1,15,Gymnasium,Karl-Theodor-von-Dalberg-Gymnasium Aschaffenbu...,Grünewaldstr. 18,63739,Aschaffenburg,www.dalberg-gymnasium.de/,/schule/0015.html
2,16,Gymnasium,Maria-Ward-Schule Mädchengymnasium der Maria-W...,Brentanoplatz 8,63739,Aschaffenburg,mws-ab.de,/schule/0016.html
3,14,Gymnasium,Friedrich-Dessauer-Gymnasium Aschaffenburg,Stadtbadstr. 4,63741,Aschaffenburg,https://www.fdg-online.de,/schule/0014.html
4,3,Gymnasium,Spessart-Gymnasium Alzenau,Brentanostr. 55,63755,Alzenau,www.spessart-gymnasium.de,/schule/0003.html
...,...,...,...,...,...,...,...,...
454,1041,Freie Waldorfschule,Priv. Waldorfschule Hof,Kolpingshöhe 3,95032,Hof,www.waldorfschule-hof.de,/schule/1041.html
455,1022,Freie Waldorfschule,Freie Waldorfschule Wernstein des Junger Main ...,Am Eisweiher 1,95336,Mainleus,www.schule-wernstein.de,/schule/1022.html
456,1037,Freie Waldorfschule,Rudolf-Steiner-Schule Coburg,Callenberg 12,96450,Coburg,www.waldorfschule-coburg.de,/schule/1037.html
457,1018,Freie Waldorfschule,Freie Waldorfschule Würzburg Schule nach der P...,Oberer Neubergweg 14,97074,Würzburg,www.waldorfschule-wuerzburg.de,/schule/1018.html


## Add the GPS coordinates

In [8]:
def address_to_gps(strasse, plz, ort):
    address_joined = ', '.join([strasse, str(plz), ort])
    geocode_result = gmaps.geocode(address_joined)
    res = (None, None)
    if geocode_result:
        res = (geocode_result[0]['geometry']['location']['lat'],
               geocode_result[0]['geometry']['location']['lng'])
    # location = geolocator.geocode(address_joined)
    return res

schools_of_interest[['latitude', 'longitude']] = pd.DataFrame(
    schools_of_interest.apply(
        lambda row: address_to_gps(row.Strasse, row.PLZ, row.Ort), axis=1
    ).tolist(), index=schools_of_interest.index)
schools_of_interest

,Schulnummer,Schultyp,Name,Strasse,PLZ,Ort,Homepage,Link,latitude,longitude
0,13,Gymnasium,Kronberg-Gymnasium Aschaffenburg,Fasaneriestr. 33,63739,Aschaffenburg,kronberg-gymnasium.de,/schule/0013.html,49.978355,9.163006
1,15,Gymnasium,Karl-Theodor-von-Dalberg-Gymnasium Aschaffenbu...,Grünewaldstr. 18,63739,Aschaffenburg,www.dalberg-gymnasium.de/,/schule/0015.html,49.974711,9.160044
2,16,Gymnasium,Maria-Ward-Schule Mädchengymnasium der Maria-W...,Brentanoplatz 8,63739,Aschaffenburg,mws-ab.de,/schule/0016.html,49.970230,9.153378
3,14,Gymnasium,Friedrich-Dessauer-Gymnasium Aschaffenburg,Stadtbadstr. 4,63741,Aschaffenburg,https://www.fdg-online.de,/schule/0014.html,49.973891,9.132169
4,3,Gymnasium,Spessart-Gymnasium Alzenau,Brentanostr. 55,63755,Alzenau,www.spessart-gymnasium.de,/schule/0003.html,50.091880,9.069813
...,...,...,...,...,...,...,...,...,...,...
454,1041,Freie Waldorfschule,Priv. Waldorfschule Hof,Kolpingshöhe 3,95032,Hof,www.waldorfschule-hof.de,/schule/1041.html,50.295264,11.917269
455,1022,Freie Waldorfschule,Freie Waldorfschule Wernstein des Junger Main ...,Am Eisweiher 1,95336,Mainleus,www.schule-wernstein.de,/schule/1022.html,50.120499,11.395754
456,1037,Freie Waldorfschule,Rudolf-Steiner-Schule Coburg,Callenberg 12,96450,Coburg,www.waldorfschule-coburg.de,/schule/1037.html,50.281828,10.923013
457,1018,Freie Waldorfschule,Freie Waldorfschule Würzburg Schule nach der P...,Oberer Neubergweg 14,97074,Würzburg,www.waldorfschule-wuerzburg.de,/schule/1018.html,49.775292,9.951761


In [9]:
# Write to file to preserve GPS coordinates
schools_of_interest.to_parquet('parquet/schools_gps.parquet', engine='pyarrow')

## Load saved school list with GPS data already added

In [10]:
schools_of_interest = pd.read_parquet('parquet/schools_gps.parquet', engine="pyarrow")

In [11]:
schools_of_interest.shape

(459, 10)

In [12]:
schools_of_interest[schools_of_interest.isna().any(axis=1)]

,Schulnummer,Schultyp,Name,Strasse,PLZ,Ort,Homepage,Link,latitude,longitude


In [13]:
schools_of_interest = schools_of_interest.dropna()
schools_of_interest.shape

(459, 10)

In [14]:
schools_of_interest['marker-symbol'] = 'star'
schools_of_interest['marker-color'] = '#FF0000'
schools_of_interest

,Schulnummer,Schultyp,Name,Strasse,PLZ,Ort,Homepage,Link,latitude,longitude,marker-symbol,marker-color
0,13,Gymnasium,Kronberg-Gymnasium Aschaffenburg,Fasaneriestr. 33,63739,Aschaffenburg,kronberg-gymnasium.de,/schule/0013.html,49.978355,9.163006,star,#FF0000
1,15,Gymnasium,Karl-Theodor-von-Dalberg-Gymnasium Aschaffenbu...,Grünewaldstr. 18,63739,Aschaffenburg,www.dalberg-gymnasium.de/,/schule/0015.html,49.974711,9.160044,star,#FF0000
2,16,Gymnasium,Maria-Ward-Schule Mädchengymnasium der Maria-W...,Brentanoplatz 8,63739,Aschaffenburg,mws-ab.de,/schule/0016.html,49.970230,9.153378,star,#FF0000
3,14,Gymnasium,Friedrich-Dessauer-Gymnasium Aschaffenburg,Stadtbadstr. 4,63741,Aschaffenburg,https://www.fdg-online.de,/schule/0014.html,49.973891,9.132169,star,#FF0000
4,3,Gymnasium,Spessart-Gymnasium Alzenau,Brentanostr. 55,63755,Alzenau,www.spessart-gymnasium.de,/schule/0003.html,50.091880,9.069813,star,#FF0000
...,...,...,...,...,...,...,...,...,...,...,...,...
454,1041,Freie Waldorfschule,Priv. Waldorfschule Hof,Kolpingshöhe 3,95032,Hof,www.waldorfschule-hof.de,/schule/1041.html,50.295264,11.917269,star,#FF0000
455,1022,Freie Waldorfschule,Freie Waldorfschule Wernstein des Junger Main ...,Am Eisweiher 1,95336,Mainleus,www.schule-wernstein.de,/schule/1022.html,50.120499,11.395754,star,#FF0000
456,1037,Freie Waldorfschule,Rudolf-Steiner-Schule Coburg,Callenberg 12,96450,Coburg,www.waldorfschule-coburg.de,/schule/1037.html,50.281828,10.923013,star,#FF0000
457,1018,Freie Waldorfschule,Freie Waldorfschule Würzburg Schule nach der P...,Oberer Neubergweg 14,97074,Würzburg,www.waldorfschule-wuerzburg.de,/schule/1018.html,49.775292,9.951761,star,#FF0000


In [42]:
import os
# import json
# import random
import requests

if not os.path.exists('europe_110.geo.json'):
    url = 'https://github.com/jupyter-widgets/ipyleaflet/raw/master/examples/europe_110.geo.json'
    r = requests.get(url)
    with open('europe_110.geo.json', 'w') as f:
        f.write(r.content.decode("utf-8"))

In [75]:
munich = (48.144227, 11.560515)
m = Map(center=munich, zoom=9, layout=Layout(width='100%', height='900px'))
m.add_control(FullScreenControl())

def lat_lon_to_marker(row):
    icon_type = {
        'Gymnasium': AwesomeIcon(
            name='fa-graduation-cap',
            marker_color='blue',
            icon_color='black',
            spin=False
        ),
        'Freie Waldorfschule': AwesomeIcon(
            name='fa-graduation-cap',
            marker_color='green',
            icon_color='black',
            spin=False
        )
    }
    marker_icon = icon_type[row.Schultyp]
    marker = Marker(icon=marker_icon, location=(row.latitude, row.longitude), draggable=False, title=row.Name)
    message = HTML()
    message.value = f'<a href="https://{row.Homepage}">{row.Name}</a>'
    marker.popup = message
    return marker

waldorf_markers = pd.DataFrame()
waldorf_markers[['markers']] = pd.DataFrame(
    schools_of_interest[schools_of_interest.Schultyp=='Freie Waldorfschule'].apply(
        lambda row: lat_lon_to_marker(row), axis=1
    ).tolist(), index=schools_of_interest[schools_of_interest.Schultyp=='Freie Waldorfschule'].index)
waldorf_cluster = MarkerCluster(
    markers=(waldorf_markers['markers'].tolist())
)
m.add_layer(waldorf_cluster);

gymnasium_markers = pd.DataFrame()
gymnasium_markers[['markers']] = pd.DataFrame(
    schools_of_interest[schools_of_interest.Schultyp=='Gymnasium'].apply(
        lambda row: lat_lon_to_marker(row), axis=1
    ).tolist(), index=schools_of_interest[schools_of_interest.Schultyp=='Gymnasium'].index)
gymnasium_cluster = MarkerCluster(
    markers=(gymnasium_markers['markers'].tolist())
)
m.add_layer(gymnasium_cluster);

display(m)

Map(center=[48.144227, 11.560515], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

## Add travel time to Munich via public transport

In [17]:
def add_travel_info(lat, lng, destination):
    # To get a typical travel time, we set departure time to next Monday morning
    today = datetime.date.today()
    next_monday_0700 = datetime.datetime.combine(
        today + datetime.timedelta(days=-today.weekday(), weeks=1),
        datetime.datetime.strptime('07:00', '%H:%M').time()
    )
    
    directions_result = gmaps.directions(origin={'lat': lat, 'lng': lng},
                                         destination=destination,
                                         mode="transit",
                                         departure_time=next_monday_0700)
    res = (None, None)
    if directions_result and directions_result[0]['legs']:
        res = (directions_result[0]['legs'][0]['duration']['value'] // 60,
               directions_result[0]['legs'][0]['distance']['value'] // 1000)
    return res      

schools_of_interest[['travel_duration_min', 'travel_distance_km']] = pd.DataFrame(
    schools_of_interest.apply(
        lambda row: add_travel_info(row.latitude, row.longitude, 'Münchner Freiheit'), axis=1
    ).tolist(), index=schools_of_interest.index)

# Write to file to travel times GPS coordinates
schools_of_interest.to_parquet('parquet/schools_gps_travel.parquet', engine='pyarrow')

## Load saved school list with GPS data and travel times

In [18]:
schools_of_interest = pd.read_parquet('parquet/schools_gps_travel.parquet', engine="pyarrow")

In [19]:
print("No public transport to destination from the following schools:")
schools_of_interest[schools_of_interest.isna().any(axis=1)]

No public transport to destination from the following schools:


,Schulnummer,Schultyp,Name,Strasse,PLZ,Ort,Homepage,Link,latitude,longitude,marker-symbol,marker-color,travel_duration_min,travel_distance_km
97,78,Gymnasium,Benediktinergymnasium Ettal,Kaiser-Ludwig-Platz 1,82488,Ettal,www.ettal-gymnasium.de,/schule/0078.html,47.570052,11.093568,star,#FF0000,NaN,NaN
106,220,Gymnasium,Schloss Neubeuern - Gymnasium Internatsschule ...,Schloßstr. 20,83115,Neubeuern,www.schloss-neubeuern.de,/schule/0220.html,47.775316,12.139214,star,#FF0000,NaN,NaN
108,162,Gymnasium,Gymnasium Landschulheim Marquartstein,Neues Schloß 1,83250,Marquartstein,www.lsh-marquartstein.de,/schule/0162.html,47.751815,12.467224,star,#FF0000,NaN,NaN
113,127,Gymnasium,Landschulheim Schloß Ising am Chiemsee des Zwe...,Schloßstr. 3,83339,Chieming,www.lshi.de,/schule/0127.html,47.923605,12.504792,star,#FF0000,NaN,NaN
118,45,Gymnasium,CJD Christophorusschulen Berchtes- gaden - Gym...,Am Dürreck 4,83471,Schönau a.Königssee,www.gymnasium-bgd.de,/schule/0045.html,47.601303,13.024647,star,#FF0000,NaN,NaN
119,319,Gymnasium,Luitpold-Gymnasium Wasserburg,Salzburger Str. 11,83512,Wasserburg a.Inn,www.gymnasium-wasserburg.de,/schule/0319.html,48.055506,12.231179,star,#FF0000,NaN,NaN
129,122,Gymnasium,Gymnasium Ergolding,Am Sportpark 8,84030,Ergolding,www.gymnasium-ergolding.de,/schule/0122.html,48.563129,12.173765,star,#FF0000,NaN,NaN
134,95,Gymnasium,Maristen-Gymnasium Furth,Klosterstr. 6,84095,Furth,www.maristen-gymnasium.de,/schule/0095.html,48.592706,12.027378,star,#FF0000,NaN,NaN
161,973,Gymnasium,Hallertau-Gymnasium Wolnzach,Anton-Dost-Str. 10,85283,Wolnzach,www.hgw.bayern,/schule/0973.html,48.591463,11.637453,star,#FF0000,NaN,NaN
193,137,Gymnasium,Gymnasium Königsbrunn,Alter Postweg 3,86343,Königsbrunn,www.gymnasiumkoenigsbrunn.de,/schule/0137.html,48.269460,10.883522,star,#FF0000,NaN,NaN


In [20]:
print(schools_of_interest.shape)
schools_of_interest = schools_of_interest.dropna()
print(schools_of_interest.shape)

(459, 14)
(418, 14)


In [21]:
schools_of_interest

,Schulnummer,Schultyp,Name,Strasse,PLZ,Ort,Homepage,Link,latitude,longitude,marker-symbol,marker-color,travel_duration_min,travel_distance_km
0,13,Gymnasium,Kronberg-Gymnasium Aschaffenburg,Fasaneriestr. 33,63739,Aschaffenburg,kronberg-gymnasium.de,/schule/0013.html,49.978355,9.163006,star,#FF0000,229.0,385.0
1,15,Gymnasium,Karl-Theodor-von-Dalberg-Gymnasium Aschaffenbu...,Grünewaldstr. 18,63739,Aschaffenburg,www.dalberg-gymnasium.de/,/schule/0015.html,49.974711,9.160044,star,#FF0000,230.0,384.0
2,16,Gymnasium,Maria-Ward-Schule Mädchengymnasium der Maria-W...,Brentanoplatz 8,63739,Aschaffenburg,mws-ab.de,/schule/0016.html,49.970230,9.153378,star,#FF0000,226.0,385.0
3,14,Gymnasium,Friedrich-Dessauer-Gymnasium Aschaffenburg,Stadtbadstr. 4,63741,Aschaffenburg,https://www.fdg-online.de,/schule/0014.html,49.973891,9.132169,star,#FF0000,232.0,385.0
4,3,Gymnasium,Spessart-Gymnasium Alzenau,Brentanostr. 55,63755,Alzenau,www.spessart-gymnasium.de,/schule/0003.html,50.091880,9.069813,star,#FF0000,261.0,404.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1034,Freie Waldorfschule,Freie Waldorfschule Erlangen,Rudolf-Steiner-Str. 2,91058,Erlangen,www.waldorfschule-erlangen.de,/schule/1034.html,49.560505,11.007372,star,#FF0000,137.0,233.0
453,1009,Freie Waldorfschule,Freie Waldorfschule Regensburg des Trägerverei...,Unterislinger Weg 32,93053,Regensburg,www.waldorfschule-regensburg.de,/schule/1009.html,48.990288,12.111778,star,#FF0000,126.0,146.0
454,1041,Freie Waldorfschule,Priv. Waldorfschule Hof,Kolpingshöhe 3,95032,Hof,www.waldorfschule-hof.de,/schule/1041.html,50.295264,11.917269,star,#FF0000,245.0,392.0
457,1018,Freie Waldorfschule,Freie Waldorfschule Würzburg Schule nach der P...,Oberer Neubergweg 14,97074,Würzburg,www.waldorfschule-wuerzburg.de,/schule/1018.html,49.775292,9.951761,star,#FF0000,194.0,309.0


In [22]:
travel_min = 60
travel_max = 120
schools_travel = schools_of_interest.loc[((schools_of_interest['travel_duration_min'] >= travel_min) & (schools_of_interest['travel_duration_min'] <= travel_max))]
schools_travel

,Schulnummer,Schultyp,Name,Strasse,PLZ,Ort,Homepage,Link,latitude,longitude,marker-symbol,marker-color,travel_duration_min,travel_distance_km
73,120,Gymnasium,Rainer-Maria-Rilke-Gymnasium Icking,Ulrichstr. 1-7,82057,Icking,www.gym-icking.de,/schule/0120.html,47.956970,11.433730,star,#FF0000,61.0,31.0
74,957,Gymnasium,Privates Gymnasium Eggenberg der St. Anna Coll...,Zeller Weg 27,82057,Icking,www.st-anna.eu,/schule/0957.html,47.974027,11.441587,star,#FF0000,68.0,29.0
75,313,Gymnasium,Gymnasium der Benediktiner Schäftlarn,Kloster Schäftlarn 2,82067,Kloster Schäftlarn,www.abtei-schaeftlarn.de,/schule/0313.html,47.978626,11.467239,star,#FF0000,69.0,28.0
89,132,Gymnasium,Landschulheim Kempfenhausen des Zweckverbandes...,Münchner Str. 49 - 63,82335,Berg,www.landschulheim-kempfenhausen.de,/schule/0132.html,47.992960,11.360220,star,#FF0000,66.0,35.0
90,980,Gymnasium,Montessori-Schule Gut Biberkor Gymnasium Berg-...,Biberkorstr. 19 - 21,82335,Berg,www.montessori-biberkor.de,/schule/0980.html,47.943470,11.379750,star,#FF0000,75.0,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,1019,Freie Waldorfschule,Freie Waldorfschule Chiemgau in Prien a.Chiems...,Bernauer Str. 34,83209,Prien a.Chiemsee,www.waldorfschule-chiemgau.de,/schule/1019.html,47.850953,12.344155,star,#FF0000,75.0,86.0
442,1050,Freie Waldorfschule,Waldorfschule Landshut (Grund- und weiterführe...,Papiererstr. 25A,84034,Landshut,www.waldorfschule-landshut.de/,/schule/1050.html,48.538770,12.144490,star,#FF0000,79.0,81.0
444,1046,Freie Waldorfschule,Freie Schule Glonntal in Baiern,Glonntalstr. 13,85625,Baiern,www.freie-schule-glonntal.de,/schule/1046.html,47.957824,11.895044,star,#FF0000,84.0,46.0
448,1035,Freie Waldorfschule,Freie Waldorfschule Landsberg,Münchener Str. 72,86899,Landsberg am Lech,www.waldorf-landsberg.de,/schule/1035.html,48.051977,10.896691,star,#FF0000,92.0,67.0


In [23]:
geo_json = to_geojson(df=df_1, lat='latitude', lon='longitude', properties=['Name', 'travel_duration_min'])
GeoJSON(geo_json)

NameError: name 'df_1' is not defined